In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from ai.TrashClassifierAE import TrashClassifier
from ai.DataLoader import DataLoader
from env import *

EPOCHS = 30
ETA = 1e-3

In [2]:
model = TrashClassifier(fine_tune=False)
model = model.cuda()

FeatureAE was loaded.


In [3]:
train_dloader = DataLoader(TRASH_TRAIN_DATA_PATH, TRASH_CAT)
# valid_dloader = DataLoader(TRASH_VALID_DATA_PATH, TRASH_CAT)

Number of data batch: 99
Number of batch: 17


In [4]:
optimizer = optim.Adam(model.parameters(), lr=ETA)
criterion = nn.NLLLoss()

In [5]:
for e in range(EPOCHS):
    
    train_loss = 0.0
    train_acc = 0.0
    valid_loss = 0.0
    valid_acc = 0.0
    
    for x_batch, y_batch in train_dloader.next_batch():
        x_batch = torch.FloatTensor(x_batch).cuda()
        y_batch = torch.LongTensor(y_batch).cuda()
        
        logps = model(x_batch)
        loss = criterion(logps, y_batch)
        
        with torch.no_grad():
            train_loss += loss.item()
            ps = torch.exp(logps)
            
            ps_k, cls_k = ps.topk(1, dim=1)
            equal = cls_k == y_batch.view(*cls_k.size())
            train_acc += torch.mean(equal.type(torch.FloatTensor))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        model.eval()
        
        #for x_batch, y_batch in valid_dloader.next_batch():
        #    x_batch = torch.FloatTensor(x_batch).cuda()
        #    y_batch = torch.LongTensor(y_batch).cuda()
            
        #    logps = model(x_batch)
        #    loss = criterion(logps, y_batch)
            
        #    valid_loss += loss.item()
            
        #    ps = torch.exp(logps)
        #    ps_k, cls_k = ps.topk(1, dim=1)
        #    equal = cls_k == y_batch.view(*cls_k.size())
        #    valid_acc += torch.mean(equal.type(torch.FloatTensor))
        
        train_loss /= len(train_dloader)
        train_acc /= len(train_dloader)
        #valid_loss /= len(valid_dloader)
        #valid_acc /= len(valid_dloader)
        
        print(f"Epochs: {e+1}/{EPOCHS}")
        print(f"Train loss: {train_loss:.8f}")
        print(f"Train acc: {train_acc:.8f}")
        #print(f"Valid loss: {valid_loss:.8f}")
        #print(f"Valid acc: {valid_acc:.8f}")
        
        model.train()

Epochs: 1/30
Train loss: 0.95999736
Train acc: 0.61764711
Epochs: 2/30
Train loss: 0.63592221
Train acc: 0.80147064
Epochs: 3/30
Train loss: 0.46344814
Train acc: 0.88235289
Epochs: 4/30
Train loss: 0.39271352
Train acc: 0.90931362
Epochs: 5/30
Train loss: 0.28759875
Train acc: 0.93382359
Epochs: 6/30
Train loss: 0.25113038
Train acc: 0.94607860
Epochs: 7/30
Train loss: 0.24193086
Train acc: 0.93382347
Epochs: 8/30
Train loss: 0.19137471
Train acc: 0.95098025
Epochs: 9/30
Train loss: 0.19977034
Train acc: 0.96078426
Epochs: 10/30
Train loss: 0.14932209
Train acc: 0.96568614
Epochs: 11/30
Train loss: 0.14176254
Train acc: 0.97058815
Epochs: 12/30
Train loss: 0.10854277
Train acc: 0.98039198
Epochs: 13/30
Train loss: 0.09819748
Train acc: 0.98284304
Epochs: 14/30
Train loss: 0.09742226
Train acc: 0.97303915
Epochs: 15/30
Train loss: 0.07802739
Train acc: 0.98774505
Epochs: 16/30
Train loss: 0.07053617
Train acc: 0.98774505
Epochs: 17/30
Train loss: 0.07162534
Train acc: 0.98774505
Epochs

In [6]:
model.save(CLF_CKPT_PATH)

Trash Classifier was saved.
